In [11]:
library(dplyr)

In [12]:
library(tidyr)

In [13]:
library(tidyverse)

In [17]:
tennis_data <- read_csv("data/player_stats.csv")

New names:
• `` -> `...1`
Rows: 500 Columns: 38
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (25): Age, Country, Plays, Wikipedia, Current Rank, Best Rank, Name, Bac...
dbl (13): ...1, Turned Pro, Seasons, Titles, Best Season, Retired, Masters, ...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [66]:
tennis_data_deleted <-tennis_data |>rename(Current_Rank = "Current Rank",Best_rank ="Best Rank",Favorite_Surface="Favorite Surface",Turned_Pro="Turned Pro",Current_Elo_Rank="Current Elo Rank",Best_Elo_Rank="Best Elo Rank",Peak_Elo_Rating="Peak Elo Rating",Last_Appearance="Last Appearance",GOAT_Rank="GOAT Rank",Best_Season="Best Season")|>
select(- c(...1,Wikipedia, Height,Masters,Birthplace,Residence,Weight,Coach,Facebook,Twitter,Nicknames,"Davis Cups",Olympics,"Grand Slams","Web Site","Team Cups","Weeks at No. 1","Tour Finals"))

In [80]:
Separated_Age_Age_When <- separate(tennis_data_deleted,Age,into= c("Age","Age_When"),sep= "\\(") |>separate(Age_When,into= c("Age_When","blank"),sep= "\\)")|>select(-blank)
#Separated Age to Age and Age_When

In [83]:
Separated_Current_Rank_Points <-separate(Separated_Age_Age_When,Current_Rank,into= c("Current_Rank","Current_Rank_Points"),sep= "\\(") |>separate(Current_Rank_Points,into= c("Current_Rank_Points","blank"),sep= "\\)")|>select(-blank)
#Separated Current_Rank to Current_Rank and Current_Rank_Points

In [89]:
Best_rank_When <-separate(Separated_Current_Rank_Points,Best_rank,into= c("Best_rank","Best_rank_When"),sep= "\\(" ) |>
separate(Best_rank_When,into= c("Best_rank_When","blank"),sep= "\\)")|>
select(-blank)
Best_rank_When

Warning message:
“Expected 2 pieces. Missing pieces filled with `NA` in 495 rows [1, 2, 3, 4, 5,
6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, ...].”


Age,Age_When,Country,Plays,Best_rank,Best_rank_When,Current_Rank_Points,Name,Backhand,Prize Money,⋯,Seasons,Active,Current_Elo_Rank,Best_Elo_Rank,Peak_Elo_Rating,Last_Appearance,Titles,GOAT_Rank,Best_Season,Retired
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<dbl>
26,25-04-1993,Brazil,Right-handed,378,NA,NA,Oscar Jose Gutierrez,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
18,22-12-2001,United Kingdom,Left-handed,326,NA,NA,Jack Draper,Two-handed,"$59,040",⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
32,03-11-1987,Slovakia,Right-handed,178,NA,NA,Lukas Lacko,Two-handed,"US$3,261,567",⋯,14,Yes,144 (1764),60 (06-02-2012),1886 (06-02-2012),18-03-2019,NA,NA,NA,NA
21,29-05-1998,"Korea, Republic of",Right-handed,236,NA,NA,Duck Hee Lee,Two-handed,"$374,093",⋯,2,Yes,NA,NA,NA,19-08-2019,NA,NA,NA,NA
27,21-10-1992,Australia,Right-handed,183,NA,NA,Bernard Tomic,Two-handed,"US$6,091,971",⋯,11,Yes,100 (1826),21 (23-03-2015),2037 (01-02-2016),14-10-2019,4,264 (6),2015,NA
22,11-02-1997,Poland,Right-handed,31,NA,NA,Hubert Hurkacz,Two-handed,"$1,517,157",⋯,5,Yes,33 (1983),33 (19-01-2020),1983 (20-01-2020),13-01-2020,1,489 (1),2019,NA
28,18-11-1991,United States,Right-handed,307,NA,NA,Sekou Bangoura,Two-handed,"$278,709",⋯,1,NA,NA,NA,NA,31-07-2017,NA,NA,NA,2017
21,12-05-1998,"Taiwan, Province of China",Right-handed,232,NA,NA,Tung Lin Wu,Two-handed,"$59,123",⋯,1,Yes,NA,NA,NA,NA,NA,NA,NA,NA
25,29-07-1994,Uzbekistan,Right-handed,417,NA,NA,Sanjar Fayziev,Two-handed,"$122,734",⋯,5,Yes,189 (1602),187 (10-01-2020),1603 (20-09-2019),NA,NA,NA,NA,NA
